In [1]:
from source.tinydb_manager import TinyDB_manager
from benchmark.LLM_benchmark import AnswerEvaluator
import pandas as pd
import os

C:\Users\Igorexy\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package punkt to C:\Users\Igorexy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Igorexy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# input_file = "llama_3.2_3b_instruct.json"
# input_file = "mistral_7b_instruct_v0.3.json"
# input_file = "qwen2.5_7b_instruct.json"
# input_file = "qwen2.5_7b_instruct_1m.json"
input_file = "qwen3_1.7b.json"

input_file = input_file.replace('-', '_')

In [9]:
LANG = "_RUS"
# LANG = ""

In [10]:
tiny_config = f"benchmark/output_LLM{LANG}/{input_file}"

tiny_manager = TinyDB_manager(tiny_config)
bench = AnswerEvaluator(data=tiny_manager.export_json())
metrics = bench.evaluate()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
merged_dict = {**metrics[0], **metrics[2]}
df = pd.DataFrame.from_dict(merged_dict, orient='index', columns=['score'])

# Путь и имя выходного файла
output_dir = f'benchmark/output_LLM{LANG}'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, input_file.replace('.json', '.xlsx'))

# Сохранение в Excel
df.to_excel(output_file)

# Чтение всех Excel-файлов из папки и объединение

In [13]:
# --- Чтение всех Excel-файлов из папки и объединение ---

output_dir = f'benchmark/output_LLM{LANG}'
# Получаем список всех .xlsx файлов
files = [f for f in os.listdir(output_dir) if f.endswith('.xlsx')]

# Загружаем каждый файл в DataFrame и переименовываем колонку в имя файла без расширения
dfs = []
for file in files:
    filepath = os.path.join(output_dir, file)
    df = pd.read_excel(filepath, index_col=0)
    df.columns = [os.path.splitext(file)[0]]  # переименование колонки
    dfs.append(df)

# Объединение всех по индексу (метрикам)
final_df = pd.concat(dfs, axis=1)


final_df.to_excel(f"benchmark/comparison_llm{LANG}.xlsx")
# Пример вывода
final_df

,qwen3_1.7b
f1_avg,0.088811
rouge_l_avg,0.126097
rouge_n_avg,0.434245
bleu_avg,0.015927
cosine_similarity_avg,0.658556
bertscore_f1_avg,0.290962
quality_score_avg,0.283958
gen_time_avg,27.402604
gen_time_median,26.965793
gen_time_95th_percentile,34.963174
